In [13]:
from annoy import AnnoyIndex
import sys

modules = '/home/dbcloud/ym/CSPG/ANNS/modules'

if modules not in sys.path:
  sys.path.append(modules)

from binary_io import *
import anns

from concurrent.futures import ThreadPoolExecutor

In [14]:
dataset = 'gud'

base = "/home/dbcloud/ym/CSPG/experiment/distribution/data/ubase.fvecs"

query = "/home/dbcloud/ym/CSPG/experiment/distribution/data/uquery.fvecs"

gt = "/home/dbcloud/ym/CSPG/experiment/distribution/data/ugt.ivecs"

output = '/home/dbcloud/ym/CSPG/experiment/output/distribution/'

In [15]:
def query_F(index, vector, k):
  return index.get_nns_by_vector(vector, k, include_distances=False)

In [16]:
def test_annoy(base, query, gt, rw, dataset):

  if rw == 'fbin_read':
    base = fbin_read(base)
    query = fbin_read(query)
  else:
    base = fvecs_read(base)
    query = fvecs_read(query)

  if dataset == 'deep1m':
    gt = ibin_read(gt)
  else:
    gt = ivecs_read(gt)

  nb, d = base.shape
  nq, ngt = gt.shape
  gt = gt.flatten().tolist()

  k = 10
  threads = 24
  # faiss.omp_set_num_threads(threads)

  print(dataset, nb, d, nq, k)

  out = open(output + dataset + '_annoy.csv', 'w')

  tm = anns.STimer()

  print('num_queries,num_tree,query_time,recall', file=out)
  for nt in [100, 200, 300, 400, 500, 800, 1600]:
    print(nt) 
    index = AnnoyIndex(d, 'euclidean')
    for i in range(nb):
      index.add_item(i, base[i])

    index.build(nt, n_jobs = threads)

    tm.reset()
    tm.start()
    with ThreadPoolExecutor() as executor:
      futures = [executor.submit(query_F, index, query[i], k) for i in range(nq)]
    tm.stop()
    results = [future.result() for future in futures]

    indices_matrix = np.zeros((nq, k), dtype=int)
    for i, indices in enumerate(results):
      indices_matrix[i, :] = indices  

    recall = anns.get_recall(k, ngt, gt, indices_matrix)
    print(f'{nq},{nt},{tm.get_time()},{recall}', file=out)

  out.close()

In [17]:
test_annoy(base, query, gt, 'fvecs_read', dataset)

gud 1000000 96 1000 10
100
200
300
400
500
800
1600
